# Processing extracted FixMe data

In [133]:
from tabulate import tabulate
from pathlib import Path
import os
import pandas as pd
import glob
import re
import json
import git
import pandas as pd
from urllib.parse import urlparse
from warnings import simplefilter

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [132]:
import sqlite3
# load cve table from /Users/guru/research/FixMe/data/FixMe.db
conn = sqlite3.connect('/Users/guru/research/FixMe/data/FixMe.db')
df_cve = pd.read_sql_query("SELECT * FROM cve;", conn)

cveIds = list(df_cve.cveId)
cveIds

['CVE-2013-2127',
 'CVE-2013-2873',
 'CVE-2013-2035',
 'CVE-2013-2023',
 'CVE-2013-2832',
 'CVE-2013-2218',
 'CVE-2013-2058',
 'CVE-2013-2234',
 'CVE-2013-2222',
 'CVE-2013-2081',
 'CVE-2013-2243',
 'CVE-2013-2146',
 'CVE-2013-2015',
 'CVE-2013-2853',
 'CVE-2013-2852',
 'CVE-2013-2079',
 'CVE-2013-2868',
 'CVE-2013-2242',
 'CVE-2013-2038',
 'CVE-2013-2080',
 'CVE-2013-2653',
 'CVE-2013-2223',
 'CVE-2013-2130',
 'CVE-2013-2833',
 'CVE-2013-2075',
 'CVE-2013-2022',
 'CVE-2013-2872',
 'CVE-2013-2126',
 'CVE-2013-2204',
 'CVE-2013-2879',
 'CVE-2013-2245',
 'CVE-2013-2496',
 'CVE-2013-2091',
 'CVE-2013-2140',
 'CVE-2013-2547',
 'CVE-2013-2117',
 'CVE-2013-2506',
 'CVE-2013-2156',
 'CVE-2013-2930',
 'CVE-2013-2160',
 'CVE-2013-2025',
 'CVE-2013-2834',
 'CVE-2013-2635',
 'CVE-2013-2232',
 'CVE-2013-2634',
 'CVE-2013-2209',
 'CVE-2013-2012',
 'CVE-2013-2546',
 'CVE-2013-2116',
 'CVE-2013-2141',
 'CVE-2013-2182',
 'CVE-2013-2244',
 'CVE-2013-2154',
 'CVE-2013-2007',
 'CVE-2013-2093',
 'CVE-2013